In [3]:
import os
import numpy as np
import pandas as pd
import nibabel as nib

from nilearn import plotting

# High-Grade Gliomas (HGG)
- Gliomas are tumors.
- HGG are tumors that grow at a high rate.

In [4]:
# HGG Train
# Creating folder path for each folder in directory
folder_contents_path = [('data/MICCAI_BraTS_2019_Data_Training/HGG/' + str(file_name)) for file_name in os.listdir('data/MICCAI_BraTS_2019_Data_Training/HGG') if 'BraTS19' in file_name]

In [5]:
# Getting all image data from HGG Training
hgg_img_data_train = []

for folder_path in folder_contents_path:
    for file in os.listdir(folder_path):
        file_path = folder_path + '/' + str(file)
        
        if '.nii.gz' in file_path:
            img = nib.load(file_path)
            array_flat = img.get_data().ravel()
            hgg_img_data_train.append(array_flat)
#             hgg_img_data_train.append(img.get_data().ravel())

In [6]:
# To see what my list looks like, delete when complete

hgg_img_data_train

[array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dt

# Low-Grade Gliomas (LGG)
- Gliomas are tumors.
- LGG are tumors that grow at a slow rate.

In [10]:
# LGG Train
# Creating folder path for each folder in directory
folder_contents_path2 = [('data/MICCAI_BraTS_2019_Data_Training/LGG/' + str(file_name)) for file_name in os.listdir('data/MICCAI_BraTS_2019_Data_Training/LGG') if 'BraTS19' in file_name]


# Getting all image data from LGG Training
lgg_img_data_train = []

for folder_path in folder_contents_path2:
    for file in os.listdir(folder_path):
        file_path = folder_path + '/' + str(file)
        
        if '.nii.gz' in file_path:
            img = nib.load(file_path)
            lgg_img_data_train.append(img.get_data().ravel())

In [11]:
# To see what my list looks like, delete when complete
lgg_img_data_train

[array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 array([0, 0, 0, ..., 0, 0, 0], dt

In [12]:
# Length of each list

print(f' Length of HGG Train List: {len(hgg_img_data_train)}')
print(f' Length of LGG Train List: {len(lgg_img_data_train)}')

 Length of HGG Train List: 1295
 Length of LGG Train List: 380


# Playground

In [13]:
img = nib.load('data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_CBICA_ANG_1/BraTS19_CBICA_ANG_1_seg.nii.gz')

In [14]:
img.shape

(240, 240, 155)

In [30]:
img.get_data().ravel().tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [45]:
pd.DataFrame(img.get_data().ravel()).T

,0,1,2,3,4,5,6,7,8,9,...,8927990,8927991,8927992,8927993,8927994,8927995,8927996,8927997,8927998,8927999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
